# Importing the data and creating the data loaders

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [3]:
# importing libraries

import glob
import sys
from zipfile import ZipFile 
import concurrent.futures
import gc
from time import time
import cv2

sys.path.insert(0,'../src/')

import PIL as pil

import pandas as pd
import numpy as np
np.random.seed(42)

import matplotlib.pyplot as plt

import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
from tqdm import tqdm
import urllib.request
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: emadonev (emadonev-xv-gimnazija) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
def clear_gpu_memory(debug:bool=True):
    """
    Clear GPU memory on all devices.
    Optinonally print the memory before and after clearing.

    Args:
        debug (bool, optional): Print memory before and after clearing. Defaults to False.
    """
    if debug:
        print("GPU memory before clearing.")
        print(f"Allocated: {torch.cuda.memory_allocated() :_}")
        print(f"Reserved: {torch.cuda.memory_reserved() :_}")
    
    # Free up memory
    gc.collect()

    # Clear CUDA memory cache
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    print("GPU memory has been cleared.")

    if debug:
        print("GPU memory after clearing.")
        print(f"Allocated: {torch.cuda.memory_allocated() :_}")
        print(f"Reserved: {torch.cuda.memory_reserved() :_}")

In [5]:
clear_gpu_memory()

GPU memory before clearing.
Allocated: 0
Reserved: 0
GPU memory has been cleared.
GPU memory after clearing.
Allocated: 0
Reserved: 0


In [6]:
from data_process import *
from cvt import *
from model_train import *

---

In [ ]:
url = "https://zenodo.org/records/3565489/files/images_gz2.zip?download=1"
urllib.request.urlretrieve(url, "../input/images_gz2.zip")

In [7]:
with ZipFile('../input/images_gz2.zip', 'r') as zip_ref:
    zip_ref.extractall(path='../input/images_gz2/')

---

In [7]:
reference_images = pd.read_csv('../input/filename_mapping.csv')

main_catalogue = pd.read_csv('../input/gz2_classes.csv')

In [8]:
reference_images.head()

,objid,sample,asset_id
0,587722981736120347,original,1
1,587722981736579107,original,2
2,587722981741363294,original,3
3,587722981741363323,original,4
4,587722981741559888,original,5


In [9]:
main_catalogue.head()

,specobjid,dr8objid,dr7objid,ra,dec,rastring,decstring,sample,gz2class,total_classifications,...,t11_arms_number_a36_more_than_4_fraction,t11_arms_number_a36_more_than_4_weighted_fraction,t11_arms_number_a36_more_than_4_debiased,t11_arms_number_a36_more_than_4_flag,t11_arms_number_a37_cant_tell_count,t11_arms_number_a37_cant_tell_weight,t11_arms_number_a37_cant_tell_fraction,t11_arms_number_a37_cant_tell_weighted_fraction,t11_arms_number_a37_cant_tell_debiased,t11_arms_number_a37_cant_tell_flag
0,1.802675e+18,NaN,588017703996096547,160.99040,11.703790,10:43:57.70,+11:42:13.6,original,SBb?t,44,...,0.225,0.225,0.225,0,10,10.0,0.250,0.250,0.250,0
1,1.992984e+18,NaN,587738569780428805,192.41083,15.164207,12:49:38.60,+15:09:51.1,original,Ser,45,...,0.000,0.000,0.000,0,0,0.0,0.000,0.000,0.000,0
2,1.489569e+18,NaN,587735695913320507,210.80220,54.348953,14:03:12.53,+54:20:56.2,original,Sc+t,46,...,0.651,0.651,0.651,0,3,3.0,0.070,0.070,0.070,0
3,2.924084e+18,1.237668e+18,587742775634624545,185.30342,18.382704,12:21:12.82,+18:22:57.7,original,SBc(r),45,...,0.071,0.071,0.071,0,6,6.0,0.429,0.429,0.429,0
4,1.387165e+18,1.237658e+18,587732769983889439,187.36679,8.749928,12:29:28.03,+08:44:59.7,extra,Ser,49,...,0.000,0.000,0.000,0,1,1.0,1.000,1.000,1.000,0


In [10]:
# create modified main catalogue
model_01_catalogue = pd.DataFrame()
model_01_catalogue['dr7ID'] = main_catalogue['dr7objid']
model_01_catalogue['class'] = main_catalogue['gz2class']
print(model_01_catalogue.shape)
model_01_catalogue.head()

(243500, 2)


,dr7ID,class
0,588017703996096547,SBb?t
1,587738569780428805,Ser
2,587735695913320507,Sc+t
3,587742775634624545,SBc(r)
4,587732769983889439,Ser


In [11]:
model_01_catalogue.drop(model_01_catalogue[model_01_catalogue['class'] == 'A'].index, inplace=True)
model_01_catalogue.shape

(243253, 2)

In [12]:
# connecting each class with the corresponding asset_id
model_01_catalogue = model_01_catalogue.merge(
    reference_images[['objid', 'asset_id']], 
    left_on='dr7ID', 
    right_on='objid', 
    how='left'
).drop(columns=['objid'])  # Drop extra 'objid' column after merging
model_01_catalogue = model_01_catalogue.sort_values(by=['asset_id']).reset_index(drop=True)

model_01_catalogue['class'] = model_01_catalogue['class'].apply(lambda x: x.replace('(', '').replace(')', '').ljust(6, '0'))
model_01_catalogue.head()

,dr7ID,class,asset_id
0,587722981741363294,Ei0000,3
1,587722981741363323,Sc0000,4
2,587722981741559888,Er0000,5
3,587722981741625481,Er0000,6
4,587722981741625484,Ei0000,7


In [13]:
# creating a label diagram table
label_diagram = pd.DataFrame(columns=['r1', 'r2', 'r3', 'r4', 'r5'])
label_diagram['asset_id'] = model_01_catalogue['asset_id']
label_diagram['r1'] = model_01_catalogue['class'].apply(choose_class1)
label_diagram['r2'] = model_01_catalogue['class'].apply(choose_class2)
label_diagram['r3'] = model_01_catalogue['class'].apply(choose_class3)
label_diagram['r4'] = model_01_catalogue['class'].apply(choose_class4)
label_diagram['r5'] = model_01_catalogue['class'].apply(choose_class5)
label_diagram.head()

,r1,r2,r3,r4,r5,asset_id
0,E,is,0,0,0,3
1,S,c,0,0,0,4
2,E,rs,0,0,0,5
3,E,rs,0,0,0,6
4,E,is,0,0,0,7


In [14]:
label_diagram[label_diagram["asset_id"]==172444]

,r1,r2,r3,r4,r5,asset_id
148289,S,b,?,ts,0,172444


In [15]:
label_diagram['r2'].value_counts()

r2
c     58314
is    50272
b     48842
rs    47458
br    15241
bn     8445
d      6429
cs     5785
a      2149
bb      318
Name: count, dtype: int64

In [14]:
label_diagram.to_csv("../input/label_diagram.csv", index=False)

In [16]:
imgs_path = '../input/images_gz2/images/'
W, H, C = 224, 224, 4

In [17]:
file_list = glob.glob(os.path.join(imgs_path, '*.jpg'))
file_list = sorted(file_list)

In [18]:
file_list = file_list[:5]

In [24]:
runs = {f: () for f in file_list}

for i in label_diagram.columns:
    if i == 'asset_id':
        continue
    # Create a mapping from asset_id to the current label value.
    label_map = label_diagram.set_index('asset_id')[i].to_dict()
    # For each file, append the label value to the tuple already stored (or create a new tuple)
    for f in file_list:
        asset_id = int(f.split('/')[-1].split('.')[0])
        label_val = label_map.get(asset_id, None)
        runs[f] = runs.get(f, ()) + (label_val,)
        
print(runs)

S
E
E
Se
E
c
rs
is
br
is
0
0
0
0
0
0
0
0
0
0
m
0
0
0
0
{'../input/images_gz2/images/100.jpg': ('S', 'c', '0', '0', 'm'), '../input/images_gz2/images/1000.jpg': ('E', 'rs', '0', '0', '0'), '../input/images_gz2/images/10000.jpg': ('E', 'is', '0', '0', '0'), '../input/images_gz2/images/100000.jpg': ('Se', 'br', '0', '0', '0'), '../input/images_gz2/images/100001.jpg': ('E', 'is', '0', '0', '0')}


---

In [26]:
run1 = [(f, runs[f][0]) for f in file_list]
print(run1)

[('../input/images_gz2/images/100.jpg', 'S'), ('../input/images_gz2/images/1000.jpg', 'E'), ('../input/images_gz2/images/10000.jpg', 'E'), ('../input/images_gz2/images/100000.jpg', 'Se'), ('../input/images_gz2/images/100001.jpg', 'E')]


In [ ]:
label_diagram['r1'].value_counts()

r1
E     103515
S      94332
Se     24004
SB     21402
Name: count, dtype: int64

In [31]:
file_list = glob.glob(os.path.join(imgs_path, '*.jpg'))
file_list = sorted(file_list)

# select files whose asset_id is the same as the one in the label_diagram
file_list = [f for f in file_list if int(f.split('/')[-1].split('.')[0]) in label_diagram['asset_id'].values]
len(file_list)

243253

In [36]:
len(file_list)

243253

In [38]:
class_mapping = {x : i for i, x in enumerate(sorted(label_diagram['r1'].unique()))}
n = 0.0411

if __name__ == '__main__':
    gmorph_d = galaxy_img_dataset(file_list, label_diagram, label_mapping='r1', class_mapping=class_mapping)
    images = [x[0] for x in gmorph_d]
    labels = [x[1] for x in gmorph_d]

    img_sub, im, labels_sub, l = train_test_split(images, labels, train_size=n, random_state=42, stratify=labels, shuffle=True)
    print(len(img_sub), len(labels_sub))

    x = np.array(img_sub)
    y = np.array(labels_sub)

    bs = 32
    # ---------
    x_train, x_rem, y_train, y_rem = train_test_split(x,y, train_size=0.7, random_state=42, stratify=y, shuffle=True)

    x_valid, x_test, y_valid, y_test = train_test_split(x_rem,y_rem, test_size=0.34, random_state=42, stratify=y_rem, shuffle=True)

    x_train, y_train, x_valid, y_valid, x_test, y_test = map(torch.tensor, (x_train, y_train, x_valid, y_valid, x_test, y_test))

    train_ds = TensorDataset(x_train, y_train)
    train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True, num_workers=16, pin_memory=True)

    valid_ds = TensorDataset(x_valid, y_valid)
    valid_dl = DataLoader(valid_ds, batch_size=bs, shuffle=True, num_workers=16,pin_memory=True)

    test_ds = TensorDataset(x_test, y_test)
    test_dl = DataLoader(test_ds, batch_size=bs, shuffle=True, num_workers=16,pin_memory=True)


: 

---

In [24]:
import gc

In [25]:
gc.collect()

674

In [27]:
epochs = 1
lr = 0.001
tmax = epochs
gmavailable() else 'cpu'
batch_size = 32
embed_size = 64available() else 'cpu'
batch_size = 32
embed_size = 64

---

In [29]:
gmorph_model = CvT(embed_size, len(class_mapping))
optimizer = torch.optim.AdamW(gmorph_model.parameters(), lr=lr, weight_decay=0.05)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, tmax, eta_min=1e-6)
loss_func = nn.CrossEntropyLoss()

results, results_class, train_pred, train_true, train_probs, valid_pred, valid_true, valid_probs = train_model(epochs, gmorph_model, train_dl, valid_dl, loss_func, optimizer, scheduler, device, save_name=f'test_shapes')

Run name: test_shapes
image shape torch.Size([32, 4, 224, 224])
aux layer selected torch.Size([32, 3, 224, 224]) torch.Size([32, 1, 64])
conv embedding shape torch.Size([32, 3136, 64])
passed through attention torch.Size([32, 3136, 64])
final out torch.Size([32, 64, 56, 56])
image shape torch.Size([32, 64, 56, 56])
conv embedding shape torch.Size([32, 784, 192])
passed through attention torch.Size([32, 784, 192])
final out torch.Size([32, 192, 28, 28])
image shape torch.Size([32, 192, 28, 28])
conv embedding shape torch.Size([32, 196, 384])
cls_token provided torch.Size([32, 197, 384])
passed through attention torch.Size([32, 197, 384])
image shape torch.Size([32, 4, 224, 224])
aux layer selected torch.Size([32, 3, 224, 224]) torch.Size([32, 1, 64])
conv embedding shape torch.Size([32, 3136, 64])
passed through attention torch.Size([32, 3136, 64])
final out torch.Size([32, 64, 56, 56])
image shape torch.Size([32, 64, 56, 56])
conv embedding shape torch.Size([32, 784, 192])
passed throu

train_F1,▁
train_acc,▁
train_loss,▁
train_precision,▁
train_recall,▁
valid_F1,▁
valid_acc,▁
valid_loss,▁
valid_precision,▁
valid_recall,▁
train_F1,0.2012


Training time = 0.0283196489016215 minutes
